In [ ]:
import numpy as np
import scipy
import itertools
import  pathlib 

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.interactive(True)
#%config InlineBackend.figure_format = 'pdf'
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from matplotlib import gridspec

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 300

plt.rcParams['figure.autolayout'] = False

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = [
    r"\usepackage{subdepth}",
    r"\usepackage{type1cm}",
    r'\usepackage{tgheros}',    # helvetica font
    r'\usepackage{sansmath}',   # math-font matching  helvetica
    r'\sansmath'                # actually tell tex to use it!
#    r'\usepackage{siunitx}',    # micro symbols
#    r'\sisetup{detect-all}',    # force siunitx to use the fonts
]  

print(plt.rcParams['figure.figsize'])


def set_size(column=0):
    if (column==0):
        plt.rcParams['figure.figsize']   = [10,6]
        plt.rcParams['axes.labelsize']   = 18 
        plt.rcParams['axes.titlesize']   = 20 
        plt.rcParams['font.size']        = 16 
        plt.rcParams['lines.linewidth']  = 2.0 
        plt.rcParams['lines.markersize'] = 8 
        plt.rcParams['legend.fontsize']  = 14 
    elif (column==1):
        plt.rcParams['figure.figsize']   = [3.5,3.5*0.6]
        plt.rcParams['axes.labelsize']   = 10 
        plt.rcParams['axes.titlesize']   = 12 
        plt.rcParams['font.size']        = 10 
        plt.rcParams['lines.linewidth']  = 1.5 
        plt.rcParams['lines.markersize'] = 10 
        plt.rcParams['legend.fontsize']  = 10 
    elif (column==2):
        plt.rcParams['figure.figsize']   = [7.0,7.0*0.6]
        plt.rcParams['axes.labelsize']   = 10 
        plt.rcParams['axes.titlesize']   = 12 
        plt.rcParams['font.size']        = 10 
        plt.rcParams['lines.linewidth']  = 2.0 
        plt.rcParams['lines.markersize'] = 10 
        plt.rcParams['legend.fontsize']  = 10 
        
set_size(column=2)

In [ ]:
dir='../data/d_tor1_tor2/'
N_windows = 1370

w_i = np.array([], dtype="int")
t   = np.array([])
q = {}
q["d"], q["tor1"], q["tor2"] = np.array([]), np.array([]), np.array([])

for window in range(0, N_windows):
    print(window, end=" ")
    try:
        c1, c2, c3, c4 = np.loadtxt(dir + str(window) + ".txt", unpack=True)
    except:
        continue

    w_i  = np.append(w_i, np.full(c1.shape, window))
    t    = np.append(t, c1)
    q["d"]    = np.append(q["d"]   , c2)
    q["tor1"] = np.append(q["tor1"], c3)
    q["tor2"] = np.append(q["tor2"], c4)

    

In [ ]:
weights = np.load("../weights/weights.npy")[]
print(weights.shape)

In [ ]:
# plot them 
plt.figure()
plt.plot(q["d"])
plt.xlabel(r"sample index")
plt.ylabel(r"r / nm")

plt.figure()
plt.plot(q["tor1"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{\Theta}^0$ / $^\circ$")

plt.figure()
plt.plot(q["tor2"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{\Theta}^1$ / $^\circ$")


# Reconstruct PMF

In [ ]:
import sys, importlib
from densitytopmf import *
importlib.reload(sys.modules['densitytopmf'])
from densitytopmf import *

In [ ]:
DG = {}
bin_centers = {}

DG_kde = {}
bin_centers_kde = {}


## $\Delta$G($\mathrm{r}$)

In [ ]:
q_t = q["d"]
_y, _x= do_pmf_1D(q_t, weights, bins=np.arange(0,5,.1))

DG["d"] = _y 
bin_centers["d"] = _x

_y, _x= do_pmf_1D_kde(q_t, weights, bins=np.arange(0,5,.1), bw_method="silverman")
DG_kde["d"] = _y
bin_centers_kde["d"] = _x

In [ ]:
x = bin_centers["d"]
y = DG["d"]
plt.plot(x, y - np.mean( y[x>3.0]), label = "histogram" )

x = bin_centers_kde["d"]
y = DG_kde["d"]
plt.plot(x, y - np.mean( y[x>3.0]), label = "kde" )

plt.ylim(-8,5)

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"$\Delta$G / (kJ/mol)")

plt.legend()

## $\Delta$G($\mathrm{r_\mathrm{min}}$)

In [ ]:
#load the rmin
dir = '../data/mindist/' 

q["mindist"]=np.full(t.shape, np.nan) 
for window in range(0, N_windows):
    try:
        c1, c2 = np.loadtxt(dir + str(window) + ".A.mindist.xvg", unpack=True, comments=["#","@"])
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["mindist"][mask] = c2         

In [ ]:
# plot them 
plt.figure()
plt.plot(q["mindist"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{r_\mathrm{min}}$ / nm")

In [ ]:
q_t = q["mindist"]
mask = np.isfinite(q_t)

_y, _x= do_pmf_1D(q_t[mask], weights[mask], bins=np.arange(0,3.5,.1))

DG["mindist"] = _y
bin_centers["mindist"] = _x

_y, _x= do_pmf_1D_kde(q_t[mask], weights[mask], bins=np.arange(0,3.5,.05), bw_method=0.05)
DG_kde["mindist"] = _y
bin_centers_kde["mindist"] = _x

In [ ]:
x = bin_centers["mindist"]
y = DG["mindist"]
plt.plot(x, y - np.mean( y[x>2.0]), label = "histogram" )

x = bin_centers_kde["mindist"]
y = DG_kde["mindist"]
plt.plot(x, y - np.mean( y[x>2.0]), label = "kde" )

plt.ylim(-12,2)

plt.xlabel(r"$\mathrm{r_\mathrm{min}}$ / nm")
plt.ylabel(r"$\Delta$G / (kJ/mol)")

plt.legend()

## $\Delta$G($\mathrm{nc}$)

In [ ]:
#load the rmin
dir = '../data/mindist/' 

q["numcont"]=np.full(t.shape, np.nan) 
for window in range(0, N_windows):
    try:
        c1, c2 = np.loadtxt(dir + str(window) + ".A.numcont.xvg", unpack=True, comments=["#","@"])
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["numcont"][mask] = c2         

In [ ]:
# plot them 
plt.figure()
plt.plot(q["numcont"])
plt.xlabel(r"sample index")
plt.ylabel(r"r / nm")

In [ ]:
q_t = q["numcont"]
mask = np.isfinite(q_t)

_y, _x= do_pmf_1D(q_t[mask], weights[mask], bins=np.arange(0,120,10))

DG["numcont"] = _y
bin_centers["numcont"] = _x

_y, _x= do_pmf_1D_kde(q_t[mask], weights[mask], bins=np.arange(0,120,1), bw_method="silverman")
DG_kde["numcont"] = _y
bin_centers_kde["numcont"] = _x

In [ ]:
x = bin_centers["numcont"]
y = DG["numcont"]
plt.plot(x, y, label = "histogram" )

x = bin_centers_kde["numcont"]
y = DG_kde["numcont"]
plt.plot(x, y, label = "kde" )


plt.xlabel(r"Number of contacts")
plt.ylabel(r"$\Delta$G / (kJ/mol)")

plt.legend()

## $\Delta$G($\mathrm{\Theta}_1$)

In [ ]:
q_t = q["tor1"]*180/np.pi
_y, _x= do_pmf_1D(q_t, weights, bins=np.arange(-180,180,10))

mask = np.isfinite(_y)
DG["tor1"] = _y[mask] 
bin_centers["tor1"] = _x[mask]

q_t = q["tor1"]*180/np.pi
_y, _x= do_pmf_1D_kde(q_t, weights, bins=np.arange(-180,180,2), bw_method="silverman")

DG_kde["tor1"] = _y
bin_centers_kde["tor1"] = _x

In [ ]:
x = bin_centers["tor1"]
y = DG["tor1"]
plt.plot(x, y, label = "histogram" )

x = bin_centers_kde["tor1"]
y = DG_kde["tor1"]
plt.plot(x, y, label = "kde" )

#plt.ylim(-8,5)

plt.xlabel(r"$\mathrm{\Theta}_1$ / $^\circ$")
plt.ylabel(r"$\Delta$G / (kJ/mol)")

plt.legend()

## $\Delta$G($\mathrm{\Theta}_2$)

In [ ]:
q_t = q["tor2"]*180/np.pi
_y, _x= do_pmf_1D(q_t, weights, bins=np.arange(-180,180,10))

mask = np.isfinite(_y)
DG["tor2"] = _y[mask] 
bin_centers["tor2"] = _x[mask]

q_t = q["tor2"]*180/np.pi
_y, _x= do_pmf_1D_kde(q_t, weights, bins=np.arange(-180,180,2), bw_method="silverman")

DG_kde["tor2"] = _y
bin_centers_kde["tor2"] = _x

In [ ]:
x = bin_centers["tor2"]
y = DG["tor2"]
plt.plot(x, y, label = "histogram" )

x = bin_centers_kde["tor2"]
y = DG_kde["tor2"]
plt.plot(x, y, label = "kde" )

#plt.ylim(-8,5)

plt.xlabel(r"$\mathrm{\Theta}_2$ / $^\circ$")
plt.ylabel(r"$\Delta$G / (kJ/mol)")

plt.legend()

## $\Delta$G(r, $\mathrm{\Theta}_1$)

In [ ]:
q1_t = q["d"]
q2_t = q["tor1"]*180/np.pi 

#histo
_z, _x, _y = do_pmf_2D(q1_t, q2_t, weights, bins=[np.arange(0,5,.2), np.arange(-180,180,10)])

DG["d","tor1"] = _z
bin_centers["d","tor1"] = [_x, _y]

#kde
_z, _x, _y = do_pmf_2D_kde(q1_t, q2_t, weights, bins=[np.arange(0,5,.2), np.arange(-180,180,10)], bw_method=0.1)

DG_kde["d","tor1"] = _z
bin_centers_kde["d","tor1"] = [_x, _y]

In [ ]:
plt.figure()

X = bin_centers["d","tor1"][0].copy()
Y = bin_centers["d","tor1"][1].copy()
Z = DG["d","tor1"].copy()  

# nor,allize with respect to bulk
mask = (X > 3) * (X < 4)
Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title("histogram")

#kde

plt.figure()

X = bin_centers_kde["d","tor1"][0].copy()
Y = bin_centers_kde["d","tor1"][1].copy()
Z = DG_kde["d","tor1"].copy()  

# nor,allize with respect to bulk
mask = (X > 3) * (X < 4)
Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title("KDE")

## $\Delta$G(r, $\mathrm{\Theta}_2$)

In [ ]:
q1_t = q["d"]
q2_t = q["tor2"]*180/np.pi 

#histo
_z, _x, _y = do_pmf_2D(q1_t, q2_t, weights, bins=[np.arange(0,5,.2), np.arange(-180,180,10)])

DG["d","tor2"] = _z
bin_centers["d","tor2"] = [_x, _y]

#kde
_z, _x, _y = do_pmf_2D_kde(q1_t, q2_t, weights, bins=[np.arange(0,5,.2), np.arange(-180,180,10)], bw_method=0.1)

DG_kde["d","tor2"] = _z
bin_centers_kde["d","tor2"] = [_x, _y]

In [ ]:
plt.figure()

X = bin_centers["d","tor2"][0].copy()
Y = bin_centers["d","tor2"][1].copy()
Z = DG["d","tor2"].copy()  

# nor,allize with respect to bulk
mask = (X > 3) * (X < 4)
Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"$\mathrm{\Theta}_2$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)

#kde

plt.figure()

X = bin_centers_kde["d","tor2"][0].copy()
Y = bin_centers_kde["d","tor2"][1].copy()
Z = DG_kde["d","tor2"].copy()  

# nor,allize with respect to bulk
mask = (X > 3) * (X < 4)
Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"$\mathrm{\Theta}_2$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)


## $\Delta$G($\mathrm{r_\mathrm{min}}$, $\mathrm{nc}$)

In [ ]:
q1_t = q["mindist"]
q2_t = q["numcont"]

mask=np.isfinite(q1_t)

#histo
_z, _x, _y = do_pmf_2D(q1_t[mask], q2_t[mask], weights[mask], bins=[np.arange(0,3.5,.1), np.arange(0,120,10)])

DG["mindist","numcont"] = _z
bin_centers["mindist","numcont"] = [_x, _y]

#kde
_z, _x, _y = do_pmf_2D_kde(q1_t[mask], q2_t[mask], weights[mask], bins=[np.arange(0,3.5,.1), np.arange(0,120,1)], bw_method=0.05)

DG_kde["mindist","numcont"] = _z
bin_centers_kde["mindist","numcont"] = [_x, _y]

In [ ]:
plt.figure()

X = bin_centers["mindist","numcont"][0].copy()
Y = bin_centers["mindist","numcont"][1].copy()
Z = DG["mindist","numcont"].copy()  

Z[Z>100]=np.nan

# nor,allize with respect to bulk
#mask = (X > 3) * (X < 4)
#Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{r_\mathrm{min}}$, / nm")
plt.ylabel(r"Number of Contacts")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)

#kde

plt.figure()

X = bin_centers_kde["mindist","numcont"][0].copy()
Y = bin_centers_kde["mindist","numcont"][1].copy()
Z = DG_kde["mindist","numcont"].copy()  

Z[Z>100]=np.nan

# nor,allize with respect to bulk
#mask = (X > 3) * (X < 4)
#Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 5, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{r_\mathrm{min}}$,/ nm")
plt.ylabel(r"Number of Contacts")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)


## $\Delta$G($\mathrm{r}$, $\mathrm{r_\mathrm{min}}$)

In [ ]:
q1_t = q["d"]
q2_t = q["mindist"]

mask=np.isfinite(q2_t)

#histo
_z, _x, _y = do_pmf_2D(q1_t[mask], q2_t[mask], weights[mask], bins=[np.arange(0,5,.2), np.arange(0,3.5,.1)])

DG["d","mindist"] = _z
bin_centers["d","mindist"] = [_x, _y]

#kde
_z, _x, _y = do_pmf_2D_kde(q1_t[mask], q2_t[mask], weights[mask], bins=[np.arange(0,5,.2), np.arange(0,3.5,.1)], bw_method=0.05)

DG_kde["d","mindist"] = _z
bin_centers_kde["d","mindist"] = [_x, _y]

In [ ]:
plt.figure()

X = bin_centers["d","mindist"][0].copy()
Y = bin_centers["d","mindist"][1].copy()
Z = DG["d","mindist"].copy()  

Z[Z>50]=np.nan

# nor,allize with respect to bulk
#mask = (X > 3) * (X < 4)
#Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, levels=np.arange(-6,41,3), alpha=1, cmap="jet");

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{r}$,/ nm")
plt.ylabel(r"$\mathrm{r_\mathrm{min}}$,/ nm")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
plt.clim(-5,45)

#kde

plt.figure()

X = bin_centers_kde["d","mindist"][0].copy()
Y = bin_centers_kde["d","mindist"][1].copy()
Z = DG_kde["d","mindist"].copy()  

#Z[Z>50]=np.nan

# nor,allize with respect to bulk
#mask = (X > 3) * (X < 4)
#Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, levels=np.arange(-6,41,3), alpha=1, cmap="jet");

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{r}$,/ nm")
plt.ylabel(r"$\mathrm{r_\mathrm{min}}$,/ nm")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-5,45)


## $\Delta$G($\mathrm{\Theta}_1$, $\mathrm{\Theta}_2$)

In [ ]:
q1_t = q["tor1"]*180/np.pi 
q2_t = q["tor2"]*180/np.pi 

#histo
_z, _x, _y = do_pmf_2D(q1_t, q2_t, weights, bins=[np.arange(-180,180,10), np.arange(-180,180,10)])

DG["tor1","tor2"] = _z
bin_centers["tor1","tor2"] = [_x, _y]

#kde
_z, _x, _y = do_pmf_2D_kde(q1_t, q2_t, weights, bins=[np.arange(-180,180,10), np.arange(-180,180,10)], bw_method=0.1)

DG_kde["tor1","tor2"] = _z
bin_centers_kde["tor1","tor2"] = [_x, _y]

In [ ]:
plt.figure()

X = bin_centers["tor1","tor2"][0].copy()
Y = bin_centers["tor1","tor2"][1].copy()
Z = DG["tor1","tor2"].copy()  

# nor,allize with respect to bulk
#mask = (X > 3) * (X < 4)
#Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{\Theta}_0$ / $^\circ$")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title(r"histogram")


#kde

plt.figure()

X = bin_centers_kde["tor1","tor2"][0].copy()
Y = bin_centers_kde["tor1","tor2"][1].copy()
Z = DG_kde["tor1","tor2"].copy()  

# nor,allize with respect to bulk
#mask = (X > 3) * (X < 4)
#Z -= np.mean(Z[mask,:], axis=0)

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{\Theta}_0$ / $^\circ$")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title(r"KDE")



## $\Delta$G($\mathrm{\Theta}_1$, $\mathrm{\Theta}_2$) in bound state

In [ ]:
q1_t = q["tor1"]*180/np.pi 
q2_t = q["tor2"]*180/np.pi 

# I define the bound state based on the followinf
q3_t = q["d"]
mask = (q3_t < 1.8)

#histo
_z, _x, _y = do_pmf_2D(q1_t[mask], q2_t[mask], weights[mask], bins=[np.arange(-180,180,10), np.arange(-180,180,10)])

DG["tor1","tor2","bound"] = _z
bin_centers["tor1","tor2","bound"] = [_x, _y]

#kde
_z, _x, _y = do_pmf_2D_kde(q1_t[mask], q2_t[mask], weights[mask], bins=[np.arange(-180,180,10), np.arange(-180,180,10)], bw_method=0.1)

DG_kde["tor1","tor2","bound"] = _z
bin_centers_kde["tor1","tor2","bound"] = [_x, _y]

In [ ]:
plt.figure()

X = bin_centers["tor1","tor2","bound"][0].copy()
Y = bin_centers["tor1","tor2","bound"][1].copy()
Z = DG["tor1","tor2","bound"].copy()  


minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{\Theta}_0$ / $^\circ$")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title(r"histogram")


#kde

plt.figure()

X = bin_centers_kde["tor1","tor2","bound"][0].copy()
Y = bin_centers_kde["tor1","tor2","bound"][1].copy()
Z = DG_kde["tor1","tor2","bound"].copy()  


minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{\Theta}_0$ / $^\circ$")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title(r"KDE")




## $\Delta$G($\mathrm{\Theta}_1$, $\mathrm{\Theta}_2$) in the unbound state

In [ ]:
q1_t = q["tor1"]*180/np.pi 
q2_t = q["tor2"]*180/np.pi 

# I define the unbound state based on the followinf
q3_t = q["d"]
mask = (q3_t > 3) * (q3_t < 4)


#histo
_z, _x, _y = do_pmf_2D(q1_t[mask], q2_t[mask], weights[mask], bins=[np.arange(-180,180,10), np.arange(-180,180,10)])

DG["tor1","tor2","unbound"] = _z
bin_centers["tor1","tor2","unbound"] = [_x, _y]

#kde
_z, _x, _y = do_pmf_2D_kde(q1_t[mask], q2_t[mask], weights[mask], bins=[np.arange(-180,180,10), np.arange(-180,180,10)], bw_method=0.1)

DG_kde["tor1","tor2","unbound"] = _z
bin_centers_kde["tor1","tor2","unbound"] = [_x, _y]

In [ ]:
plt.figure()

X = bin_centers["tor1","tor2","unbound"][0].copy()
Y = bin_centers["tor1","tor2","unbound"][1].copy()
Z = DG["tor1","tor2","unbound"].copy()  


minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{\Theta}_0$ / $^\circ$")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title(r"histogram")

#kde

plt.figure()

X = bin_centers_kde["tor1","tor2","unbound"][0].copy()
Y = bin_centers_kde["tor1","tor2","unbound"][1].copy()
Z = DG_kde["tor1","tor2","unbound"].copy()  

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{\Theta}_0$ / $^\circ$")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title(r"KDE")




## $\Delta$G($\mathrm{\Theta}_1$, $\mathrm{\Theta}_2$)$_\mathrm{bound}$ - $\Delta$G($\mathrm{\Theta}_1$, $\mathrm{\Theta}_2$)$_\mathrm{unbound}$

In [ ]:
plt.figure()

X = bin_centers["tor1","tor2","unbound"][0].copy()
Y = bin_centers["tor1","tor2","unbound"][1].copy()
Z = DG["tor1","tor2","bound"] - DG["tor1","tor2","unbound"]

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{\Theta}_0$ / $^\circ$")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title(r"histogram")

#kde

plt.figure()

X = bin_centers_kde["tor1","tor2","unbound"][0].copy()
Y = bin_centers_kde["tor1","tor2","unbound"][1].copy()
Z = DG_kde["tor1","tor2","bound"] - DG_kde["tor1","tor2","unbound"]

minx, maxx, miny, maxy = X.min(), X.max(), Y.min(), Y.max() 
aspect= (maxx-minx) / (maxy-miny)
plt.gca().set_aspect(aspect)

#plt.imshow(DG.T, origin="lower", extent=(minx, maxx, miny, maxy), aspect= (maxx-minx) / (maxy-miny), interpolation="bicubic" )
_X, _Y = np.meshgrid(X, Y)
plt.contourf(_X, _Y, Z.T, 20, alpha=1);

plt.gca().xaxis.set_ticks_position('bottom')

plt.xlabel(r"$\mathrm{\Theta}_0$ / $^\circ$")
plt.ylabel(r"$\mathrm{\Theta}_1$ / $^\circ$")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\Delta$G / (kJ/mol)")
#plt.clim(-15,10)
plt.title(r"KDE")




## Contact Frequency($\mathrm{r}$, i)

In [ ]:
#load the rmin
dir = '../data/contacts/' 

q["contactsA"]=np.full([t.shape[0], 35], np.nan) 
for window in range(0, N_windows):
    try:
        c = np.loadtxt(dir + str(window) + ".A.mask.xvg", comments=["#","@"])
        c1 = c[:,0]
        c2 = c[:,1:]
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["contactsA"][mask,:] = c2         

In [ ]:
#load the rmin
dir = '../data/contacts/' 

q["contactsB"]=np.full([t.shape[0], 35], np.nan) 
for window in range(0, N_windows):
    try:
        c = np.loadtxt(dir + str(window) + ".B.mask.xvg", comments=["#","@"])
        c1 = c[:,0]
        c2 = c[:,1:]
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["contactsB"][mask,:] = c2  

In [ ]:
# plot them 
plt.figure()

q_t = q["contactsA"].copy()
q_t [q_t==0] = np.nan

plt.matshow(q_t, aspect=q_t.shape[1]/q_t.shape[0])
plt.xlabel(r"contact")
plt.ylabel(r"sample index")

plt.figure()

q_t = q["contactsB"].copy()
q_t [q_t==0] = np.nan

plt.matshow(q_t, aspect=q_t.shape[1]/q_t.shape[0])
plt.xlabel(r"contact")
plt.ylabel(r"sample index")

In [ ]:
q1_t = q["contactsA"]
q2_t = q["d"]

mask = np.isfinite(q1_t[:,0])

i=0

A , bin_centers = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))

for i in range(1,35):
    y, x = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))
    A = np.vstack([A, y])

plt.figure()

#A[A<0.00001] = np.nan
minx, maxx, miny, maxy = bin_centers.min(), bin_centers.max(), .5, 35.5 

plt.imshow(A, origin="lower",
           extent = (minx, maxx, miny, maxy), 
           aspect = (maxx-minx) / (maxy-miny),
           interpolation="gaussian",
           cmap="hot_r"
          )
cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"Contact Frequency")

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"Residue No.")

In [ ]:
q1_t = q["contactsB"]
q2_t = q["d"]

mask = np.isfinite(q1_t[:,0])

i=0

A , bin_centers = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))

for i in range(1,35):
    y, x = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))
    A = np.vstack([A, y])

plt.figure()

#A[A<0.00001] = np.nan
minx, maxx, miny, maxy = bin_centers.min(), bin_centers.max(), .5, 35.5 

plt.imshow(A, origin="lower",
           extent = (minx, maxx, miny, maxy), 
           aspect = (maxx-minx) / (maxy-miny),
           interpolation="gaussian",
           cmap="hot_r"
          )
cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"Contact Frequency")

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"Residue No.")

## Contact Distance($\mathrm{r}$, i)

In [ ]:
#load the rmin
dir = '../data/mindist/' 

q["mindistresA"]=np.full([t.shape[0], 35], np.nan) 
for window in range(0, N_windows):
    try:
        c = np.loadtxt(dir + str(window) + ".A.mindistres.xvg", comments=["#","@"])
        c1 = c[:,0]
        c2 = c[:,1:]
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["mindistresA"][mask,:] = c2         

In [ ]:
#load the rmin
dir = '../data/mindist/' 

q["mindistresB"]=np.full([t.shape[0], 35], np.nan) 
for window in range(0, N_windows):
    try:
        c = np.loadtxt(dir + str(window) + ".B.mindistres.xvg", comments=["#","@"])
        c1 = c[:,0]
        c2 = c[:,1:]
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["mindistresB"][mask,:] = c2         

In [ ]:
# plot them 
plt.figure()

q_t = q["mindistresA"].copy()
q_t [q_t==0] = np.nan

plt.matshow(q_t, aspect=q_t.shape[1]/q_t.shape[0])
plt.xlabel(r"mindistres")
plt.ylabel(r"sample index")

# plot them 
plt.figure()

q_t = q["mindistresB"].copy()
q_t [q_t==0] = np.nan

plt.matshow(q_t, aspect=q_t.shape[1]/q_t.shape[0])
plt.xlabel(r"mindistres")
plt.ylabel(r"sample index")

In [ ]:
q1_t = q["mindistresA"]
q2_t = q["d"]

mask = np.isfinite(q1_t[:,0])

i=0

A , bin_centers = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))

for i in range(1,35):
    y, x = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))
    A = np.vstack([A, y])

plt.figure()

#A[A<0.00001] = np.nan
minx, maxx, miny, maxy = bin_centers.min(), bin_centers.max(), .5, 35.5 

plt.imshow(A, origin="lower", extent = (minx, maxx, miny, maxy), aspect = (maxx-minx) / (maxy-miny) , interpolation="none" )
cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\mathrm{r_{min}}$ / nm")

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"Residue No.")

In [ ]:
q1_t = q["mindistresB"]
q2_t = q["d"]

mask = np.isfinite(q1_t[:,0])

i=0

A , bin_centers = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))

for i in range(1,35):
    y, x = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))
    A = np.vstack([A, y])

plt.figure()

#A[A<0.00001] = np.nan
minx, maxx, miny, maxy = bin_centers.min(), bin_centers.max(), .5, 35.5 

plt.imshow(A, origin="lower", extent = (minx, maxx, miny, maxy), aspect = (maxx-minx) / (maxy-miny) , interpolation="none" )
cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\mathrm{r_{min}}$ / nm")

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"Residue No.")

## Pair Distances($\mathrm{r}$, ij)

In [ ]:
#load the rmin
dir = '../data/pairdist/' 

q["pairdist"]=np.full([t.shape[0], 1225], np.nan) 
for window in range(0, N_windows):
    try:
        c = np.loadtxt(dir + str(window) + ".dist.xvg", comments=["#","@"])
        c1 = c[:,0]
        c2 = c[:,1:]
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["pairdist"][mask,:] = c2         

In [ ]:
# plot them 
plt.figure()

q_t = q["pairdist"].copy()
q_t [q_t==0] = np.nan

plt.matshow(q_t, aspect=q_t.shape[1]/q_t.shape[0])
plt.xlabel(r"pairdist")
plt.ylabel(r"sample index")

In [ ]:
q1_t = q["pairdist"]
q2_t = q["d"]

mask = np.isfinite(q1_t[:,0])

i=0

A , bin_centers = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))

for i in range(1,1225):
    y, x = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask], bins=np.arange(0.4,5,0.1))
    A = np.vstack([A, y])

In [ ]:
plt.figure()

#A[A<0.00001] = np.nan
minx, maxx, miny, maxy = bin_centers.min(), bin_centers.max(), .5, 1295.5 

plt.imshow(A, origin="lower", extent = (minx, maxx, miny, maxy), aspect = (maxx-minx) / (maxy-miny) , interpolation="none" )
cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"Pairdist")

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"Residue No.")

In [ ]:
B = A.reshape(35,35,-1)
print(B.shape, B.max(), B.min())

In [ ]:
for i in range(B.shape[2]-1,-1,-1):
    #print(i)
    if bin_centers[i] >= 2.25:
        continue
    
    plt.figure()

    Z = B[:,:,i].copy()
    #Z[Z>2] = np.nan

    minx, maxx, miny, maxy = 0.5, 35.5, 0.5, 35.5 
    aspect= (maxx-minx) / (maxy-miny)
    plt.gca().set_aspect(aspect)

    plt.imshow(Z, origin="lower", extent = (minx, maxx, miny, maxy),
               aspect = (maxx-minx) / (maxy-miny),
               interpolation="gaussian", cmap="gist_stern")
    #_X, _Y = np.meshgrid(np.arange(1,36), np.arange(1,36))
    #plt.contourf(_X, _Y, Z.T, 5, alpha=1, cmap="jet_r");

    plt.gca().xaxis.set_ticks_position('bottom')
    
    cbar=plt.colorbar(fraction=0.046, pad=0.04)
    cbar.set_label (r"$\mathrm{d_{ij}}$ / nm")
    
    plt.clim(0.4,2)

    plt.xlabel(r"Residue No. (chain A)")
    plt.ylabel(r"Residue No. (chain B)")
    plt.title(r"$\mathrm{r} = $" + '{0:.2f}'.format(bin_centers[i]) + " $\mathrm{nm}$" )

# Testing the distribution of $\varphi$ based on 3 random vectors

$$\varphi=\operatorname{atan2}(|\mathbf{u}_2|\,\mathbf{u}_1 \cdot(\mathbf{u}_2 \times \mathbf{u}_3) , (\mathbf{u}_1 \times \mathbf{u}_2) \cdot (\mathbf{u}_2 \times \mathbf{u}_3))$$


This is the proof of concept about the distribution of the torsion angle defined based on 3 vectors or 4 points 

In [ ]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

N_phi = 100000
phi = np.full(N_phi, np.nan)

for i in range (0, N_phi):
    u1 = np.random.rand(3)
    u2 = np.random.rand(3)
    u3 = np.random.rand(3)

    u2_norm = np.sum(u2**2.0)**0.5

    term0 = np.dot(u2_norm*u1, np.cross(u2, u3))
    term1 = np.dot(np.cross(u1, u2), np.cross(u2, u3))

    phi[i] = np.arctan2(term0, term1)*180/np.pi
    
phi    

In [ ]:
data = phi
bins = np.arange(-180,180, 5)
weights_ = np.ones(phi.shape) 

hist, bin_edges = np.histogram(data, bins=bins, weights=weights_, density=True)


#########################################
Temperature = 310
# Boltzmann constant in kJ/(mol.K)
kB = 0.0083144621
beta = 1.0/(kB*Temperature)
########################################

kBT = kB * Temperature #kJ/mol
DG_ = -kBT*np.log(hist)

bin_centers_=0.5*(bin_edges[1:]+bin_edges[:-1])



In [ ]:
plt.figure()
plt.plot(bin_centers_, hist)
plt.xlabel(r"$\mathrm{\Theta}_1$ / $^\circ$")
plt.ylabel(r"$\rho$")
plt.title(r"distribution")

plt.figure()
plt.plot(bin_centers_, DG_)
plt.xlabel(r"$\mathrm{\Theta}_1$ / $^\circ$")
plt.ylabel(r"$\Delta$G / (kJ/mol)")
plt.title(r"PMF")

This time, make the vectors from 4 random points

In [ ]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

N_phi = 100000
phi = np.full(N_phi, np.nan)

for i in range (0, N_phi):
    # 4-points
    p1 = np.random.rand(3)
    p2 = np.random.rand(3)
    p3 = np.random.rand(3)
    p4 = np.random.rand(3)

    u1 = p1 - p2
    u2 = p3 - p2
    u3 = p4 - p3

    u2_norm = np.sum(u2**2.0)**0.5

    term0 = np.dot(u2_norm*u1, np.cross(u2, u3))
    term1 = np.dot(np.cross(u1, u2), np.cross(u2, u3))

    phi[i] = np.arctan2(term0, term1)*180/np.pi
    
phi    

In [ ]:
data = phi
bins = np.arange(-180,180, 5)
weights_ = np.ones(phi.shape) 

hist, bin_edges = np.histogram(data, bins=bins, weights=weights_, density=True)

kBT = kB * Temperature #kJ/mol
DG_ = -kBT*np.log(hist)

bin_centers_ = 0.5*(bin_edges[1:]+bin_edges[:-1])



In [ ]:
plt.figure()
plt.plot(bin_centers_, hist)
plt.xlabel(r"$\mathrm{\Theta}_1$ / $^\circ$")
plt.ylabel(r"$\rho$")
plt.title(r"distribution")

plt.figure()
plt.plot(bin_centers_, DG_)
plt.xlabel(r"$\mathrm{\Theta}_1$ / $^\circ$")
plt.ylabel(r"$\Delta$G / (kJ/mol)")
plt.title(r"PMF")